In [127]:
import os
import getpass

os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key: ")

In [128]:
os.environ["PINECONE_API_KEY"] = getpass.getpass("Pinecone API Key:")

In [129]:
from uuid import uuid4
unique_id = uuid4().hex[0:8]

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = f"Mecainic - {unique_id}"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_API_KEY"] = getpass.getpass("Enter your LangSmith API key:")

In [130]:
from langchain.document_loaders import PyMuPDFLoader
from dotenv import load_dotenv
import os

load_dotenv()

# car_manual = PyMuPDFLoader('docs/owner_manual.pdf')
car_manual = PyMuPDFLoader(os.environ.get('pdfurl'))

In [131]:
car_manual_data = car_manual.load()
print(car_manual_data[5])

page_content='4\n \nResetting user data for change of\nownership\n129\nResetting settings in the centre\ndisplay\n129\nTable showing centre display settings\n130\nDriver profiles\n131\nSelecting driver profile\n132\nRenaming a driver profile\n132\nProtect driver profile\n133\nLinking remote control key to\ndriver profile\n133\nResetting settings in the driver\nprofiles\n134\nMessage in centre display\n135\nManaging messages in the centre\ndisplay\n135\nHandling a message saved from\nthe centre display\n136\nHead-up display*\n137\nActivating and deactivating the\nhead-up display*\n138\nSettings for head-up display*\n139\nVoice recognition\n140\nUsing voice recognition\n141\nControlling a telephone with voice\nrecognition\n143\nVoice control of radio and media\n144\n \nSettings for voice recognition\n145\nLIGHTING\nLighting control\n148\nAdjusting light functions via the\ncentre display\n150\nAdapting the headlamp pattern\nfrom the headlamps\n150\nAdjusting headlamp level\n150\nPosition 

In [132]:
partial_car_manual_data = car_manual_data

print(partial_car_manual_data[0])

page_content="XC60\nOWNER'S MANUAL\n" metadata={'source': 'https://az685612.vo.msecnd.net/pdfs/20w17/XC60_OwnersManual_MY21_en-GB_TP32005/XC60_OwnersManual_MY21_en-GB_TP32005.pdf', 'file_path': 'https://az685612.vo.msecnd.net/pdfs/20w17/XC60_OwnersManual_MY21_en-GB_TP32005/XC60_OwnersManual_MY21_en-GB_TP32005.pdf', 'page': 0, 'total_pages': 720, 'format': 'PDF 1.4', 'title': '', 'author': '', 'subject': '', 'keywords': '', 'creator': '', 'producer': 'GPL Ghostscript 9.06', 'creationDate': "D:20200421104728+02'00'", 'modDate': "D:20200421104728+02'00'", 'trapped': ''}


In [133]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
import tiktoken

def tiktoken_len(text):
    tokens = tiktoken.encoding_for_model("gpt-3.5-turbo").encode(
        text,
    )
    return len(tokens)

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 400,
    chunk_overlap = 50,
    length_function = tiktoken_len,
)

car_manual_chunks = text_splitter.split_documents(partial_car_manual_data)
car_manual_chunks[:5]

[Document(page_content="XC60\nOWNER'S MANUAL", metadata={'source': 'https://az685612.vo.msecnd.net/pdfs/20w17/XC60_OwnersManual_MY21_en-GB_TP32005/XC60_OwnersManual_MY21_en-GB_TP32005.pdf', 'file_path': 'https://az685612.vo.msecnd.net/pdfs/20w17/XC60_OwnersManual_MY21_en-GB_TP32005/XC60_OwnersManual_MY21_en-GB_TP32005.pdf', 'page': 0, 'total_pages': 720, 'format': 'PDF 1.4', 'title': '', 'author': '', 'subject': '', 'keywords': '', 'creator': '', 'producer': 'GPL Ghostscript 9.06', 'creationDate': "D:20200421104728+02'00'", 'modDate': "D:20200421104728+02'00'", 'trapped': ''}),
 Document(page_content="VÄLKOMMEN!\nWe hope you will enjoy many years of driving pleasure in your Volvo.\nThe car has been designed for the safety and comfort of you and\nyour passengers. Volvo strives to design one of the safest cars in the\nworld. Your Volvo is also designed to meet applicable safety and\nenvironmental requirements.\nTo increase your enjoyment of your Volvo, we recommend that you\nread the ins

In [134]:
max_chunk_length = 0

for chunk in car_manual_chunks:
    max_chunk_length = max(max_chunk_length, tiktoken_len(chunk.page_content))

max_chunk_length

400

In [136]:
from pinecone import Pinecone, PodSpec
pinecone_client = Pinecone()

pinecone_client.create_index(
    name=os.environ.get('index'),
    dimension=1536,
    metric="cosine",
    spec=PodSpec(
        environment="gcp-starter"
    )
)

while not pinecone_client.describe_index(os.environ.get('index')).status['ready']:
    print("loading")

print(f"index created: {os.environ.get('index')}")

index created: mechainic


In [137]:
from langchain.vectorstores import Pinecone
from langchain_openai import OpenAIEmbeddings

embedding_model = OpenAIEmbeddings(model="text-embedding-3-small")

vector_store = Pinecone.from_documents(car_manual_chunks, embedding_model, index_name=os.environ.get('index'))

In [138]:
retriever = vector_store.as_retriever()

In [139]:
from langchain_core.prompts import ChatPromptTemplate

RAG_PROMPT = """

CONTEXT:
{context}

QUERY:
{question}

You are a car specialist and can only provide your answers from the context. 

Don't tell in your response that you are getting it from the context.

"""

rag_prompt = ChatPromptTemplate.from_template(RAG_PROMPT)

In [140]:
from langchain_openai import ChatOpenAI
model = ChatOpenAI(model="gpt-3.5-turbo")

In [141]:
from operator import itemgetter
from langchain.schema.runnable import RunnablePassthrough

mecanic_qa_chain = (
    {"context": itemgetter("question") | retriever, "question": itemgetter("question")}
    | RunnablePassthrough.assign(context=itemgetter("context"))
    | {"response": rag_prompt | model, "context": itemgetter("context")}
)


In [142]:
response = mecanic_qa_chain.invoke({"question" : "What is the purpose of Event Data Recorder?"})

In [143]:
response["response"]

AIMessage(content='The purpose of the Event Data Recorder (EDR) in the vehicle is to register and record data related to traffic accidents or collision-like situations, such as when the airbag deploys or the vehicle strikes an obstacle in the road. The data is recorded to increase understanding of how vehicle systems work in these types of situations.', response_metadata={'token_usage': {'completion_tokens': 65, 'prompt_tokens': 1936, 'total_tokens': 2001}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_3bc1b5746c', 'finish_reason': 'stop', 'logprobs': None})

In [144]:
mecanic_qa_chain.invoke({"question" : "what is the primary purpose of Event Data Recorder (EDR)?"})

{'response': AIMessage(content='The primary purpose of the Event Data Recorder (EDR) is to register and record data related to traffic accidents or collision-like situations, such as when the airbag deploys or the vehicle strikes an obstacle in the road.', response_metadata={'token_usage': {'completion_tokens': 45, 'prompt_tokens': 1939, 'total_tokens': 1984}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_3bc1b5746c', 'finish_reason': 'stop', 'logprobs': None}),
 'context': [Document(page_content="The EDR in this vehicle is designed to record\ndata related to the following in the event of\ntraffic accidents or collision-like situations:\n•\nHow the various systems in the car\nworked\n•\nWhether the driver and passenger seat-\nbelts were fastened/tensioned\n•\nThe driver's use of the accelerator or brake\npedal\n•\nThe travel speed of the vehicle\nThis information can help us understand the\ncircumstances in which traffic accidents, inju-\nries and damage occur. The EDR only 

In [ ]:
dev_validation_questions = [
    "what is the primary purpose of Event Data Recorder?",
    "how do you manage the centre display?",
    "how do I adjust the door mirrors?"
    "how do I lower the backrests in the rearseat?"
    "how do I lock and unlock the car?"
    "what is a care key?"
]

dev_key_words_it_should_mention = [
    {"must_contain": ["EDR"]},
    {"must_contain": ["centre display", "display"]},
    {"must_contain": ["door mirrors", "door", "mirrors"]},
    {"must_contain": ["backrests", "rearseat"]},
    {"must_contain": ["lock", "unlock"]},
    {"must_contain": ["care", "key"]},
]

In [ ]:
from langsmith import Client

langsmith_client = Client()
dataset_name =  f"Mecainic - evaluation dataset {uuid4.hex[0:8]}";

dataset = langsmith_client.create_dataset(
    dataset_name= dataset_name,
    description="Questions on a instruction manual for use on validating mecainic RAG"
)

langsmith_client.create_examples(
    inputs=[{"question": question} for question in dev_validation_questions],
    outputs= dev_key_words_it_should_mention,
    dataset_id=dataset.id
)


In [ ]:
from langsmith.evaluation import EvaluationResult, run_evaluator

@run_evaluator
def must_contain(run, example) -> EvaluationResult:
    prediction = run.outputs.get("output") or ""
    required = example.output.get("must_contain") or []
    score = all(phrase in prediction for phrase in required)
    return EvaluationResult(key="must_contain", score=score)


In [ ]:
from langchain.smith import RunEvalConfig, run_on_dataset

eval_configuration = RunEvalConfig(
    custom_evaluators=[must_contain],
    evaluators=[
        "criteria",
        "cot_qa",
    ],
)

In [ ]:
langsmith_client.run_on_dataset(
    dataset_name=dataset_name,
    llm_or_chain_factory=mecanic_qa_chain,
    evaluation=eval_configuration,
    verbose=True,
    project_name=f"Mecainic RAG evaluation - {uuid4.hex[0:8]}",
    project_metadata={"version": "1.0.0"}
)
